In [64]:
#API_KEY = <add your api key>

In [66]:
import pandas as pd
from mistralai import Mistral
from tqdm import tqdm
import time
import ast

# Load the dataset
questions_file = "questions.csv"
df = pd.read_csv(questions_file)

# Initialize Mistral client
client = Mistral(api_key=API_KEY)

# Define a function to determine the domain/specialty of the question
def identify_specialty(body, a, b, c, d, e):
    prompt = (
        "Identify the medical specialty of the following question based on the question text and possible answers.\n"
        f"{body}\n"
        f"A: {a}\n"
        f"B: {b}\n"
        f"C: {c}\n"
        f"D: {d}\n"
        f"E: {e}\n"
        "Provide the name of the medical specialty."
    )
    while True:
        try:
            response = client.chat.complete(
                model="open-mistral-nemo",
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    },
                ],
                temperature=0.0
            )
            specialty = response.choices[0].message.content.strip()
            return specialty
        except Exception as e:
            print(f"Error: {e}. Retrying after a delay...")
            time.sleep(5)

# Define a prompt generation function for each domain expert
def domain_expert_prompt(specialty, body, a, b, c, d, e, role):
    # Define different characteristics for each doctor
    if role == "Dr. Alpha":
        characteristic = "You are a prudent expert. You prefer combinations that are safe and have high confidence, even if they are not the most aggressive treatments."
    elif role == "Dr. Beta":
        characteristic = "You are a conservative expert. You prefer combinations that minimize potential risks and prioritize patient safety."
    elif role == "Dr. Gamma":
        characteristic = "You are an evidence-based expert. You strictly adhere to clinical guidelines and assign confidence based on strong medical evidence."
    else:
        characteristic = "You are a general medical expert."

    return (
        f"As a {specialty} domain expert, {role}, answer the following question. {characteristic} Provide three possible answer combinations with their confidence percentages. Only return the JSON in the format \{{\"combination\": \"confidence\"}}: for example {{\"A,C\": 60, \"A,B\": 50, \"A\": 40}}. Do not include any explanations or justifications."
        f"{body}\n"
        f"A: {a}\n"
        f"B: {b}\n"
        f"C: {c}\n"
        f"D: {d}\n"
        f"E: {e}\n"
        "Provide the answers in the format: for example {\"A,C\": 60, \"A,B\": 50, \"A\": 40}."
    )

# Generate answers for each question
answers = []
detailed_responses = []

print("Starting the processing of questions...")
for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing questions"):
    # Step 1: Identify the specialty of the question
    print(f"Processing question ID {_}: Identifying specialty...")
    specialty = identify_specialty(
        row["question"],
        row["answer_A"],
        row["answer_B"],
        row["answer_C"],
        row["answer_D"],
        row["answer_E"]
    )
    print(f"Specialty identified: {specialty}")

    # Step 2: Get responses from three domain experts
    responses = []
    for role in ["Dr. Alpha", "Dr. Beta", "Dr. Gamma"]:
        print(f"Getting response from {role}... (without justification)")
        prompt = domain_expert_prompt(
            specialty,
            row["question"],
            row["answer_A"],
            row["answer_B"],
            row["answer_C"],
            row["answer_D"],
            row["answer_E"],
            role
        )
        while True:
            try:
                chat_response = client.chat.complete(
                    model="ft:mistral-large-latest:64a72f25:20241012:1862d7db",
                    messages=[
                        {
                            "role": "user",
                            "content": prompt
                        },
                    ],
                    temperature=0.1
                )
                response = chat_response.choices[0].message.content.strip()
                responses.append(response)
                detailed_responses.append({"role": role, "response": response})
                print(f"Response from {role}: {response}")
                break
            except Exception as e:
                print(f"Error: {e}. Retrying after a delay...")
                time.sleep(5)

    # Step 3: Combine responses to determine the most likely answer combinations
    print("Combining responses to determine the final answer...")
    answer_probabilities = {}
    for response in responses:
        try:
            response_dict = ast.literal_eval(response)  # Safely parse the response assuming it's a valid Python dictionary
            for combination, probability in response_dict.items():
                if combination in answer_probabilities:
                    answer_probabilities[combination] += probability
                else:
                    answer_probabilities[combination] = probability
        except Exception as e:
            # Ask an LLM to parse the JSON
            try:
                parse_prompt = (
                    "Extract the valid JSON dictionary from the following response. "
                    "If the response is malformed, correct it and return only the JSON.\n"
                    f"Response: {response}"
                )
                parse_response = client.chat.complete(
                    model="ft:mistral-large-latest:64a72f25:20241012:1862d7db",
                    messages=[
                        {
                            "role": "user",
                            "content": parse_prompt
                        },
                    ],
                    temperature=0.0
                )
                response_dict = ast.literal_eval(parse_response.choices[0].message.content.strip())
                for combination, probability in response_dict.items():
                    if combination in answer_probabilities:
                        answer_probabilities[combination] += probability
                    else:
                        answer_probabilities[combination] = probability
            except Exception as e2:
                print(f"Error parsing response with LLM: {e2}. Consulting a human expert (doctor) for response: {response}")

    # Select the combination with the highest combined probability
    final_answer = max(answer_probabilities, key=answer_probabilities.get)

    answers.append(final_answer)
    response_summary = {role_response['role']: role_response['response'] for role_response in detailed_responses[-3:]}
    
    print(f"Final answer for question ID {_}: {final_answer}")

# Create output DataFrame and save it to CSV
print("Saving the answers to output.csv...")
output_df = pd.DataFrame({
    "Answer": answers,
    "Dr_Alpha_Response": [resp.get("response") for resp in detailed_responses if resp.get("role") == "Dr. Alpha"],
    "Dr_Beta_Response": [resp.get("response") for resp in detailed_responses if resp.get("role") == "Dr. Beta"],
    "Dr_Gamma_Response": [resp.get("response") for resp in detailed_responses if resp.get("role") == "Dr. Gamma"]
})
output_df.index.name = "id"
output_df.to_csv("multi-step.csv")
print("Processing complete. Output saved.")

Starting the processing of questions...


Processing questions:   0%|                                                                                          | 0/103 [00:00<?, ?it/s]

Processing question ID 0: Identifying specialty...
Specialty identified: The medical specialty associated with this question is **Pediatrics**. Here's why:

1. The question is about a child's rash (exanthème roséoliforme) and fever (fébrile), which are common pediatric concerns.
2. The listed options (A, B, C, D, E) are all pediatric conditions or syndromes:
   - A: Exanthem subitum (Roseola infantum)
   - B: Epidemic megalerythema (Erythema infectiosum)
   - C: Rubéole (German measles)
   - D: Mononucléose infectieuse (Infectious mononucleosis)
   - E: Syndrome de Kawasaki (Kawasaki disease)
3. The question asks for the main etiologies (causes) of this type of rash in children, which is a typical pediatric concern.

Therefore, the medical specialty is Pediatrics.
Getting response from Dr. Alpha... (without justification)
Response from Dr. Alpha: {"A,C": 80, "A,D": 60, "A,E": 50}
Getting response from Dr. Beta... (without justification)
Response from Dr. Beta: {"A,C": 60, "A,D": 50, "A

Processing questions:   1%|▊                                                                                 | 1/103 [00:07<12:27,  7.33s/it]

Response from Dr. Gamma: {"A,C": 70, "A,D": 60, "A,E": 50}
Combining responses to determine the final answer...
Final answer for question ID 0: A,C
Processing question ID 1: Identifying specialty...
Specialty identified: The medical specialty for this question is Cardiology. Here's why:

* The question is about heart failure (insuffisance cardiaque), which is a key area of focus in cardiology.
* The options discuss various signs and symptoms related to heart failure, such as heart murmurs (A, B), jugular vein distention (C), peripheral edema (D), and lung crackles (E), all of which are assessed and interpreted by cardiologists.
* Cardiology is the specialty that deals with the diagnosis and treatment of diseases of the heart and blood vessels.

Therefore, the correct answer is Cardiology.
Getting response from Dr. Alpha... (without justification)
Response from Dr. Alpha: {"A,B,E": 90, "A,B": 80, "A,E": 70}
Getting response from Dr. Beta... (without justification)
Error: API error occur

...

Response from Dr. Gamma: {"A,C": 80, "A,B,C": 70, "C": 60}
Combining responses to determine the final answer...
Final answer for question ID 100: A,C
Processing question ID 101: Identifying specialty...
Error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}. Retrying after a delay...
Specialty identified: The medical specialty for this question is **Neurology** or **Emergency Medicine**. Here's why:

1. The question involves a patient presenting with a traumatic brain injury (TBI), which is a neurological emergency.
2. The patient's history of anticoagulation therapy with warfarin (antivitamine K) and the need to manage her coagulation status in the context of a TBI are key aspects of the case.
3. The mention of INR (International Normalized Ratio) and the presence of petechiae on the brain scan indicate a need for hemostatic management, which is a crucial aspect of neurological and emergency care.
4. The possible answers involve decisions regarding anticoagul

Processing questions:  99%|███████████████████████████████████████████████████████████████████████████████▏| 102/103 [32:21<00:19, 19.35s/it]

Response from Dr. Gamma: {"A,C": 60, "A,E": 50, "A": 40}
Combining responses to determine the final answer...
Final answer for question ID 101: A,C
Processing question ID 102: Identifying specialty...
Specialty identified: The medical specialty for this question is Geriatric Medicine or Gerontology. The question specifically asks about severe malnutrition in the elderly, indicating a focus on the health and care of older adults. The other possible answers (A, B, C, D, E) are all indicators related to malnutrition, which is a common issue in geriatric care.
Getting response from Dr. Alpha... (without justification)
Error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}. Retrying after a delay...
Response from Dr. Alpha: {"A,B": 80, "A,C": 70, "A,D": 60}
Getting response from Dr. Beta... (without justification)
Response from Dr. Beta: {"A,B": 70, "A,C": 60, "A,D": 50}
Getting response from Dr. Gamma... (without justification)
Error: API error occurred: Status 42

Processing questions: 100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [32:38<00:00, 19.02s/it]

Response from Dr. Gamma: {"A,B": 80, "A,C": 70, "A,B,C": 60}
Combining responses to determine the final answer...
Final answer for question ID 102: A,B
Saving the answers to output.csv...
Processing complete. Output saved.


In [69]:
output_df[['Answer']].to_csv("nemo+ft:mistral-large-latest:64a72f25:20241012:1862d7db+nemo+multi-step-submission.csv")

- it gives a bit of explainability too
- it looks like un parcours de traitement